## Inverse Scan Augmentation

the inversion of the segmentation masks of the augmented CT-scans back to the affine of the ground truth segmentation masks. As a prerequisite, the user must provide the ground truth CT scan files as well as the corresponding ground truth segmentation masks.

Please note that the function get_matching_segmentation_mask() might fail based on the naming convention employed in the augmentation pipeline.


In [ ]:
import torch
import torchio as tio
import os 
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
import logging
import traceback

Set global variables

In [ ]:
image_records = []
list_of_ids = []
inversion_list =[]
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

#### pick_files():
The following method opens a series of file dialogs for the user to select directories and returns the selected paths.

    The function prompts the user to select the following directories:
    1. The folder containing the segmentation masks of the augmented CT-scans.
    2. The folder containing the inversion information .pth file.
    3. The output directory where processed files will be saved.
    4. The directory for storing the logging file.

    Returns:
        tuple: A tuple containing four paths (transformed_ct_segmentation_masks_path, pth_file, output_path, logging_output_path)

In [ ]:
def pick_files():
    root = tk.Tk()
    root.withdraw()  # Hide the main tkinter window
    transformed_ct_segmentation_masks_path = filedialog.askdirectory(title="Select the folder with the SEGMENTATION MASKS of THE AUGMENTED CT-SCANS")  # Show a directory selection dialog
    print(f"Selected folder: {transformed_ct_segmentation_masks_path}")
    pth_file = filedialog.askopenfilename(title="Select the INVERSION INFORMATION .PTH FILE")
    print(f"Selected file: {pth_file}")
    output_path = filedialog.askdirectory(title="Select the OUTPUT LOCATION for the OUTPUT FILES")
    print(f"Selected folder: {output_path}")
    logging_output_path = filedialog.askdirectory(title="Select the OUTPUT LOCATION for the LOGGING FILE")
    print(f"Selected folder: {logging_output_path}")
    return transformed_ct_segmentation_masks_path, pth_file, output_path, logging_output_path

#### get_nifti_file_paths():
Retrieves the file paths of NIfTI files in the specified directory.

This function searches for files with '.nii' or '.nii.gz' extensions
in the given directory and returns their full paths.

Args:
    directory_path (str): The path to the directory containing NIfTI files.

Returns:
    list of str: A list of full file paths for each NIfTI file found in the directory.

In [ ]:
def get_nifti_file_paths(directory_path):
    return [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.nii') or f.endswith('.nii.gz')]


#### get_matching_segmentation_mask():
Finds the matching .pth file tuple containing the path of the corresponding ground truth segmentation mask from a list based on the filename extracted from the given path.

This function extracts a filename from the `TS_GTCT_SM_path` and searches 
through `pth_touples` to find a tuple where the filename matches the basename of the path to the corresponding ground truth segmentation mask.

Args:
    TS_GTCT_SM_path (str): The file path of a transformed label map.
    pth_touples (list of tuple): A list of tuples, where each tuple's first 
                                     element is a ground truth segmentation mask path.

Returns:
    tuple: The first matching tuple from `pth_touples` where the extracted 
           filename matches the basename of the ground truth segmentation mask path.

In [ ]:
def get_matching_segmentation_mask(TS_GTCT_SM_path, pth_touples):
    file_name = os.path.basename(TS_GTCT_SM_path).split("SM")[1].split(".nii.gz")[0]
    for touple in pth_touples:
        if file_name == os.path.basename(touple[0].split(".nii.gz")[0])[2:]:
            return touple


#### apply_inverse_augmentation()
Applies the inverse transformation from a .pth file to a transformed label map.

This function takes a file path of a transformed label map and a tuple containing
the path of the corresponding ground truth segmentation mask and the inverse
transformation from a .pth file. It applies the inverse transformation to the
transformed label map and returns a tuple containing the path of the ground
truth segmentation mask and the transformed label map with the inverse
transformation applied.

Args:
    TS_ACT_SM_path (str): The file path of a transformed label map.
    path_inversion_touple (tuple): A tuple containing the path of the
        corresponding ground truth segmentation mask and the inverse
        transformation from a .pth file.

Returns:
    tuple: A tuple containing the path of the ground truth segmentation
        mask and the transformed label map with the inverse transformation
        applied.

In [2]:
def apply_inverse_augmentation(TS_ACT_SM_path, path_inversion_touple):
    subject = tio.Subject(image=tio.LabelMap(TS_ACT_SM_path))
    transformation = path_inversion_touple[1]
    inverse_transformation = transformation(subject)
    return (path_inversion_touple[0], inverse_transformation)

#### save_as_nifti():
Saves the inverse transformation of a label map as a NIfTI file to disk.

Args:
    inversion_touple (tuple): A tuple containing the original file path and the inverse transformation that was applied to the label map.
    output_path (str): The directory where the transformed label map will be saved.

Returns:
    None

In [ ]:
def save_as_nifti(inversion_touple, output_path):
    filename=os.path.basename(inversion_touple[0]).split("SM_")[1].split(".nii.gz")[0] + "_INVERTED.nii.gz"
    output_path = get_unique_file_path(f"{output_path}/{filename}")
    inversion_touple[1]['image'].save(output_path)

#### get_unique_file_path():

Returns a unique file path if the given file path already exists.

If the given file path does not exist, it is returned as is.
If the file exists, a counter is added to the file name until a unique file is found.

Parameters:

    file_path : str
        The file path to check

Returns:

    str
        The unique file path

In [ ]:
def get_unique_file_path(file_path):
    if not os.path.exists(file_path):
        return file_path

    # If the file exists, modify the name to make it unique
    base, _ = os.path.splitext(file_path)
    counter = 1

    # Add a counter to the file name until a unique file is found
    while os.path.exists(file_path):
        file_path = f"{base}_({counter}).nii.gz"
        counter += 1

    return file_path

### Running the different methods

In [ ]:
# Pick files
transformed_ct_segmentation_masks_path, pth_file, output_path, logging_output_path = pick_files()
# Get nifti file paths
TS_ACT_SM_file_list = get_nifti_file_paths(transformed_ct_segmentation_masks_path)
# Load .pth file
pth_touples = torch.load(pth_file, weights_only=False)
# Set up logging
logging_output_path = os.path.join(logging_output_path, f'inversion_error.log')
logging.basicConfig(
    filename=logging_output_path,
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
    )
# For each TS-ACT-SM file
for TS_ACT_SM_file in tqdm(TS_ACT_SM_file_list, total=len(TS_ACT_SM_file_list), desc="Applying inverses"):
    try:
        # Find matching ground truth segmentation mask and inversion transformation
        inversion_touple = get_matching_segmentation_mask(TS_ACT_SM_file, pth_touples)
        # Apply inverse
        inverse_transformation = apply_inverse_augmentation(TS_ACT_SM_file,inversion_touple)
        # Save inverse tranmsformation as nifti
        save_as_nifti(inverse_transformation, output_path)
    except Exception as e:
        logging.error(f"Error processing {TS_ACT_SM_file}: {e}, continuing...")
        logging.error(traceback.format_exc())
        continue